In [1]:
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

In [13]:
# things we need for Tensorflow
import numpy
import tflearn
import tensorflow
import random
import pickle

In [3]:
# import our chat-bot intents file
import json
with open('intents.json') as file:
    data = json.load(file)
#try:
    #with open("data.pickle", "rb") as f:
        #words, labels, training, output = pickle.load(f)

In [4]:
print(data["intents"])

[{'tag': 'greeting', 'patterns': ['Hi', 'How are you', 'Is anyone there?', 'Hello', 'Good day'], 'responses': ['Hello, thanks for visiting', 'Good to see you again', 'Hi there, how can I help?'], 'context_set': ''}, {'tag': 'goodbye', 'patterns': ['Bye', 'See you later', 'Goodbye'], 'responses': ['See you later, thanks for visiting', 'Have a nice day', 'Bye! Come back again soon.']}, {'tag': 'thanks', 'patterns': ['Thanks', 'Thank you', "That's helpful"], 'responses': ['Happy to help!', 'Any time!', 'My pleasure']}, {'tag': 'hours', 'patterns': ['What hours are you open?', 'What are your hours?', 'When are you open?'], 'responses': ["We're open every day 9am-9pm", 'Our hours are 9am-9pm every day']}, {'tag': 'mopeds', 'patterns': ['Which mopeds do you have?', 'What kinds of mopeds are there?', 'What do you rent?'], 'responses': ['We rent Yamaha, Piaggio and Vespa mopeds', 'We have Piaggio, Vespa and Yamaha mopeds']}, {'tag': 'payments', 'patterns': ['Do you take credit cards?', 'Do you

In [14]:
#try:
    #with open("data.pickle", "rb") as f:
        #words, labels, training, output = pickle.load(f)

In [15]:
words = []
labels = []
docs_x = []
docs_y = []
#ignore_words = ['?']
# loop through each sentence in our intents patterns
for intent in data['intents']:
    for pattern in intent['patterns']:
        # tokenize each word in the sentence
        wrds= nltk.word_tokenize(pattern)
        # add to our words list
        words.extend(wrds)
        # add to documents in our corpus
        docs_x.append(wrds)
        docs_y.append(intent["tag"])
        # add to our classes list
        if intent['tag'] not in labels:
            labels.append(intent['tag'])

In [16]:
# stem and lower each word and remove duplicates
words = [stemmer.stem(w.lower()) for w in words for w in words if w != "?"]
words = sorted(list(set(words)))

# remove duplicates
labels = sorted(labels)

#print (len(documents), "documents")
#print (len(classes), "classes", classes)
#print (len(words), "unique stemmed words", words)

In [17]:
training = []
output = []
out_empty = [0 for _ in range(len(labels))]
for x, doc in enumerate(docs_x):
    bag = []
    wrds = [stemmer.stem(w.lower()) for w in doc]
    for w in words:
            if w in wrds:
                bag.append(1)
            else:
                bag.append(0)
    output_row = out_empty[:]
    output_row[labels.index(docs_y[x])] = 1
    training.append(bag)
    output.append(output_row)

In [18]:
training = numpy.array(training)
output = numpy.array(output)
with open("data.pickle", "wb") as f:
        pickle.dump((words, labels, training, output), f)

In [19]:
#with open("data.pickle", "wb") as f:
    #pickle.dump((words, labels, training, output), f)
tensorflow.reset_default_graph()
#tensorflow.reset_default_graph()

net = tflearn.input_data(shape=[None, len(training[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(output[0]), activation="softmax")
net = tflearn.regression(net)

model = tflearn.DNN(net)

try:
    model.load("model.tflearn")
except:
    model.fit(training, output, n_epoch=1000, batch_size=8, show_metric=True)
    model.save("model.tflearn")

In [20]:
def bag_of_words(s, words):
    bag = [0 for _ in range(len(words))]

    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(word.lower()) for word in s_words]

    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1
            
    return numpy.array(bag)

In [21]:
def chat():
    print("Start talking with the bot (type quit to stop)!")
    while True:
        inp = input("You: ")
        if inp.lower() == "quit":
            break

        results = model.predict([bag_of_words(inp, words)])
        results_index = numpy.argmax(results)
        tag = labels[results_index]

        for tg in data["intents"]:
            if tg['tag'] == tag:
                responses = tg['responses']

        print(random.choice(responses))

chat()

Start talking with the bot (type quit to stop)!
You: hi
Good to see you again
You: how
Have a nice day
You: quit
